In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:


pip install torch_geometric

In [ ]:
!pip install sklearn

In [ ]:
import pandas as pd
import torch
import numpy as np
import os
import random
import sklearn
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout

from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from torch_geometric.utils import to_dense_adj
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_max_pool
from torch.nn import Sequential, Linear, ReLU

## Seizure Data Object list:

In [ ]:

node_dir = "/content/drive/MyDrive/graph_data/Features/absolute/Seizure"
edge_dir = "/content/drive/MyDrive/graph_data/Adjacency Matrix/Seizure"
seizure_data_list = []

try:
  node_files = [f for f in os.listdir(node_dir) if f.endswith(".csv")]
  edge_files = [f for f in os.listdir(edge_dir) if f.endswith(".npy")]


  if len(node_files) != len(edge_files):
      raise ValueError("The number of node feature files and edge index files do not match.")

  for node_file in node_files:

      load_path = os.path.join(node_dir, node_file)
      node_df = pd.read_csv(load_path, header=None)
      node_df = node_df.drop(node_df.index[0])
      node_df = node_df.drop(node_df.columns[0], axis=1)
      node_features = node_df.values.astype(float)
      x = torch.tensor(node_features, dtype=torch.float)


      edge_file = node_file.replace(".csv", ".npy")
      if edge_file not in edge_files:
          raise FileNotFoundError(f"Edge file for {node_file} not found.")
      edge_load_path = os.path.join(edge_dir, edge_file)


      edge_index = np.load(edge_load_path)
      adj_t = torch.tensor(edge_index, dtype=torch.long)

      edge_index = adj_t.nonzero().t().contiguous()

      # adjacency = to_dense_adj(edge_index)[0]
      # adjacency += torch.eye(len(adjacency))
      # edge_index = adjacency


      y = torch.tensor([1], dtype=torch.long)  # label: 1 is Seizure
      data = Data(x=x, edge_index=edge_index, y=y)
      seizure_data_list.append(data)

  print(f"Loaded {len(seizure_data_list)} data objects.")

except Exception as e:
  print(f"Following Error {e} occured")


In [ ]:
seizure_data_list[15].edge_index

In [ ]:
import itertools
edges = list(itertools.permutations(range(18),2))
full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous()
print(full_adj)

In [ ]:

node_dir = "/content/drive/MyDrive/graph_data/Features/absolute/NonSeizure"
edge_dir = "/content/drive/MyDrive/graph_data/Adjacency Matrix/NonSeizure"
interictal_data_list = []

try:
  node_files = [f for f in os.listdir(node_dir) if f.endswith(".csv")]
  edge_files = [f for f in os.listdir(edge_dir) if f.endswith(".npy")]


  if len(node_files) != len(edge_files):
      raise ValueError("The number of node feature files and edge index files do not match.")

  for node_file in node_files:

      load_path = os.path.join(node_dir, node_file)
      node_df = pd.read_csv(load_path, header=None)
      node_df = node_df.drop(node_df.index[0])
      node_df = node_df.drop(node_df.columns[0], axis=1)
      node_features = node_df.values.astype(float)
      x = torch.tensor(node_features, dtype=torch.float)


      edge_file = node_file.replace(".csv", ".npy")
      if edge_file not in edge_files:
          raise FileNotFoundError(f"Edge file for {node_file} not found.")
      edge_load_path = os.path.join(edge_dir, edge_file)


      edge_index = np.load(edge_load_path)
      adj_t = torch.tensor(edge_index, dtype=torch.long)

      edge_index = adj_t.nonzero().t().contiguous()
      # adjacency = to_dense_adj(edge_index)[0]
      # adjacency += torch.eye(len(adjacency))
      # edge_index = adjacency

      y = torch.tensor([0], dtype=torch.long)  # label: 0 is Non Seizure
      data = Data(x=x, edge_index=edge_index, y=y)
      interictal_data_list.append(data)

  print(f"Loaded {len(interictal_data_list)} data objects.")

except Exception as e:
  print(f"Following Error {e} occured")

In [ ]:
data_list = seizure_data_list + interictal_data_list

In [ ]:
random.shuffle(data_list)

In [ ]:
from torch_geometric.loader import DataLoader
train_data = data_list[:int(len(data_list)*0.82)]
val_data= data_list[int(len(data_list)* 0.82): int(len(data_list)* 0.9)]
test_data = data_list[int(len(data_list)* 0.9):]

In [ ]:
len(train_data)

In [ ]:
len(val_data)

In [ ]:

len(test_data)

## Model

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Graph Isomorphism Network

###  Graph Embedding is created using concatention from the global pool of the 3 GINConv Layers

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GlobalAttention
from torch.nn import Linear
class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h,):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        #self.conv4 = GINConv(
         #    Sequential(Linear(dim_h//4, dim_h//8), BatchNorm1d(dim_h//8), ReLU(),
          #      Linear(dim_h//8, dim_h//8), ReLU()))


        self.lin1 = Linear(dim_h*2,dim_h)
        self.lin2 = Linear(dim_h, 1)

    def forward(self, x, edge_index, batch):
        import itertools
        edges = list(itertools.permutations(range(18),2))
        full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous().to(device)

        h1 = self.conv1(x, edge_index)
        h1 = F.elu(h1)
        h2 = self.conv2(h1, edge_index)

      #  h4 = self.conv4(h3, edge_index)

        m1 = global_max_pool(h1, batch)
        a1 = global_mean_pool(h1, batch)
        m2 = global_max_pool(h2, batch)
        a2 = global_mean_pool(h2,batch)
        # m3 = global_max_pool(h3, batch)
        # a3 = global_mean_pool(h3, batch)


        h = torch.cat((a1,a2), dim = 1)
       # h = global_mean_pool(h2,batch)



        #h3 = global_mean_pool(h3, batch)
        # h = torch.cat9((a_pool1, a_pool2, a_pool3), dim = 1)


        h = self.lin1(h)
        h = F.relu(h)
        h = F.dropout(h, p = 0.5 , training = self.training)
        h = self.lin2(h)
        #print(len(torch.sigmoid(h)))
        return torch.sigmoid(h)


### GRAPH EMBEDDING: Global Mean Pooling


In [ ]:

class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin2 = Linear(dim_h*3, 1)
        self.lin   = Linear(dim_h, 1)

    def forward(self, x, edge_index, batch):
        h1 = self.conv1(x, edge_index)
        h1 = F.relu(h1)
        h2 = self.conv2(h1, edge_index)
        h2 = F.relu(h2)
        h3 = self.conv3(h2, edge_index)


        hG = global_mean_pool(h3, batch)

        x = F.dropout(hG,p = 0.5, training = self.training)
        x = self.lin(x)

        #print(len(torch.sigmoid(h)))
        return torch.sigmoid(x)



## Graph Convolutional Network

In [ ]:
class GCN(torch.nn.Module):
  def __init__(self,dim_h):
    super(GCN, self).__init__()
    name = "GCN"
    self.conv1 = GCNConv(num_node_features, dim_h)
    self.conv2 = GCNConv(dim_h, dim_h//2)
    self.conv3 = GCNConv(dim_h//2, dim_h//4)
    #self.conv4 = GCNConv(dim_h, dim_h)
    self.lin1 = Linear((dim_h//4+dim_h+dim_h//2), dim_h)
    self.lin2 = Linear(dim_h, 1)


  def forward(self, x, edge_index, batch):
    import itertools
    edges = list(itertools.permutations(range(18),2))
    full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous().to(device)
    h1 = self.conv1(x, edge_index)
    x = F.relu(h1)
    h2 = self.conv2(x, edge_index)
    x = F.relu(h2)
    h3 = self.conv3(x, full_adj)

    m1 = global_max_pool(h1, batch)
    a1 = global_mean_pool(h1, batch)
    m2 = global_max_pool(h2, batch)
    a2 = global_mean_pool(h2,batch)
    a3 = global_mean_pool(h3,batch)

    #h = global_mean_pool(h3, batch)

    h = torch.cat((a1,a2,a3), dim = 1)
    #print(f"The shape after concat {h.shape}",)

    h = self.lin1(h)
    h = F.relu(h)
    h = F.dropout(h, p = 0.5 , training = self.training)
    h = self.lin2(h)
    #print(len(torch.sigmoid(h)))
    return torch.sigmoid(h)





# GCN with Global Mean Pooling:

In [ ]:
class GCN_m(torch.nn.Module):
  def __init__(self,dim_h):
    super(GCN_m, self).__init__()

    self.conv1 = GCNConv(num_node_features, dim_h)
    self.conv2 = GCNConv(dim_h, dim_h//2)
    self.conv3 = GCNConv(dim_h//2, dim_h//4)
    #self.conv4 = GCNConv(dim_h, dim_h)
    self.lin   = Linear(dim_h//4, 1)



  def forward(self,x, edge_index, batch):
    import itertools
    edges = list(itertools.permutations(range(18),2))
    full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous().to(device)
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = self.conv2(x,edge_index)
    x = F.relu(x)
    x = self.conv3(x ,full_adj)
    #x = F.relu(x)
   # x = self.conv4(x, edge_index)
    #print("After Layer 3")
    #print(x.shape)

    hG = global_mean_pool(x,batch)

    x = F.dropout(hG,p = 0.5, training = self.training)
    x = self.lin(x)
    #print(len(x))

    return torch.sigmoid(x)
    #print(len(x))



## Graph Attention Networks

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GATv2Conv, global_mean_pool, global_max_pool

class GAT(torch.nn.Module):
    def __init__(self, dim_h, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(num_node_features, dim_h, heads=heads)
        self.gat2 = GATv2Conv(dim_h * heads, dim_h, heads=1)


        self.lin = Linear(dim_h+dim_h*heads, 1)

    def forward(self, x, edge_index, batch):
        import itertools
        edges = list(itertools.permutations(range(18),2))
        full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous().to(device)

        h = F.dropout(x, p=0.5, training=self.training)
        h1 = self.gat1(h, edge_index)
        h1 = F.relu(h1)
        h1 = F.dropout(h1, p=0.5, training=self.training)
        h2 = self.gat2(h1, edge_index)

       # m1 = global_max_pool(h1, batch)
        a1 = global_mean_pool(h1, batch)
      # m2 = global_max_pool(h2, batch)
        a2 = global_mean_pool(h2, batch)

        #h = torch.cat((m1, a1, m2, a2), dim=1)
        h = torch.cat((a1,a2),dim = 1)
       # h  = global_mean_pool(h2,batch)

        x = F.dropout(h, p=0.5, training=self.training)
        x = self.lin(x)
        return torch.sigmoid(x)


## GraphSAGE


In [ ]:
from binascii import a2b_hex

from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self, dim_h):
        super().__init__()
        self.sage1 = SAGEConv(num_node_features, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_h//2)
        self.sage3 = SAGEConv(dim_h//2, dim_h//4)
        self.lin = Linear((dim_h//4), 1)

    def forward(self, x, edge_index, batch):
        import itertools
        edges = list(itertools.permutations(range(18),2))
        full_adj = torch.tensor(edges,dtype = torch.long).t().contiguous().to(device)

        h1 = self.sage1(x, edge_index)
        h = F.elu(h1)
        h = F.dropout(h, p=0.5, training=self.training)
        h2 = self.sage2(h, edge_index)
        h = F.elu(h2)
        h = F.dropout(h, p=0.5, training=self.training)
        h3 = self.sage3(h,full_adj)


        a1 = global_mean_pool(h1,batch)
        m1 = global_max_pool(h1,batch)
        a2 = global_mean_pool(h2,batch)
        m2 = global_max_pool(h2,batch)

        #h = torch.cat((a1,a2), dim = 1)
        h = global_mean_pool(h3, batch)
        x = F.dropout(h, p=0.5, training=self.training)
        x = self.lin(x)
        return torch.sigmoid(x)


### GRAPH EMBEDDING: Concatenation of the addition of the 3 GCN Layers

In [ ]:
# class GCN(torch.nn.Module):
#     def __init__(self, dim_h):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(num_node_features, dim_h)
#         self.conv2 = GCNConv(dim_h, dim_h)
#         self.conv3 = GCNConv(dim_h, dim_h)
#         self.lin1 = Linear(dim_h * 3, dim_h * 3)
#         self.lin2 = Linear(dim_h * 3, 1)

#     def forward(self, x, edge_index, batch):
#         h1 = self.conv1(x, edge_index)
#         h1 = F.relu(h1)
#         h2 = self.conv2(h1, edge_index)
#         h2 = F.relu(h2)
#         h3 = self.conv3(h2, edge_index)

#         # Apply global pooling on the output of each convolutional layer
#         h1_pool = global_add_pool(h1, batch)
#         h2_pool = global_add_pool(h2, batch)
#         h3_pool = global_add_pool(h3, batch)

#         # Concatenate the pooled features from each layer
#         h = torch.cat((h1_pool, h2_pool, h3_pool), dim=1)

#         # Apply linear layers and dropout
#         h = self.lin1(h)
#         h = h.relu()
#         h = F.dropout(h, p=0.5, training=self.training)
#         h = self.lin2(h)

#         return torch.sigmoid(h)


In [ ]:
def Roc_curve(labels, preds, model_name):
  fpr, tpr, thresholds = roc_curve(labels, preds)
  plt.figure(figsize = (4,4))
  plt.plot(fpr, tpr)
  plt.title(f"{model_name}")
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])

In [ ]:
def loader(batch_size):
  train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
  val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = True)
  test_loader = DataLoader(test_data, batch_size = 1 , shuffle = True)

  return train_loader, val_loader, test_loader

In [ ]:
def save_classification_report_txt(report_str, model_name, file_path):
    with open(file_path, 'w') as f:
        f.write(f"Classification Report for {model_name}:\n")
        f.write(report_str)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def Confusion_matrix(confmat, model_name):


    fig, ax = plt.subplots(figsize=(4, 4))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')


    ax.set_xticklabels(['']+['Not Seizure', 'Seizure'])
    ax.set_yticklabels(['']+['Not Seizure', 'Seizure'])

    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f"{model_name}")
    # ax.xaxis.set_label_position('top')

    plt.tight_layout()

    plt.show()



def c_matplot(model, test_loader, j):
  all_labels = []
  all_preds = []
  model_name = type(model).__name__
  #model.eval()
  #with torch.no_grad():
  for data in test_loader:
    data.x = data.x.to(device)
    data.edge_index = data.edge_index.to(device)
    data.batch = data.batch.to(device)
    data.y = data.y.to(device)

    out = model(data.x, data.edge_index, data.batch)
    out = out.squeeze()
    out = (out >= 0.5).int()
    out = out.view(-1).detach().cpu().numpy()
    all_labels.extend(data.y.cpu().numpy())
    all_preds.extend(out)

  clf_rp = classification_report(all_labels, all_preds)
  save_classification_report_txt(clf_rp, model_name, f'/content/drive/MyDrive/graph_data/Classification_reports/exp{j}_{model_name}.txt')
  conf = confusion_matrix(y_true = all_labels, y_pred = all_preds)
  roc_curve = Roc_curve(all_labels, all_preds, model_name)
  return Confusion_matrix(conf, model_name),print(clf_rp), roc_curve


In [ ]:
# def train(model, loader, lr, epochs):
#   criterion = torch.nn.BCELoss()
#   optimizer = torch.optim.Adam(model.parameters(), lr = lr)
#   epochs = epochs
#   model.train()

#   train_accuracies = []
#   val_accuracies = []
#   for epoch in range(epochs+1):
#     total_loss = 0
#     acc = 0
#     val_loss = 0
#     val_acc = 0

#     for data in loader:
#       data.x = data.x.to(device)
#       data.edge_index = data.edge_index.to(device)
#       data.y = data.y.to(device).float()
#       data.batch = data.batch.to(device)
#       #data.y = data.y.view(-1,1)

#       optimizer.zero_grad()
#       out = model(data.x, data.edge_index, data.batch)
#       out = out.view(-1)
#       #print("Before view", out)
#       loss = criterion(out, data.y)
#       total_loss += loss/ len(loader)
#       acc += accuracy(out, data.y)/len(loader)
#       loss.backward()
#       optimizer.step()

#       val_loss, val_acc = test(model, val_loader)
#     val_accuracies.append(val_acc)

#    # total_loss /= len(loader)
#    # acc /= len(loader)

#     train_accuracies.append(acc)
#     if(epoch % 5 == 0):
#       print(f'Epoch {epoch:>3} | Train Loss: {total_loss:.2f} | Train Acc: {acc*100:>.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')
#     #print(len(val_accuracies), len(train_accuracies))

#   model_name = type(model).__name__
#   plt.figure(figsize=(10, 5))
#   plt.plot(range(epochs + 1), train_accuracies, label='Training Accuracy')
#   plt.plot(range(epochs + 1), val_accuracies, label='Validation Accuracy')
#   plt.xlabel('Epochs')
#   plt.ylabel('Accuracy')
#   plt.legend()
#   plt.title(f'Training vs. Validation Accuracy of {model_name}')
#   plt.show()

#   return model, train_accuracies, val_accuracies

# @torch.no_grad()
# def test(model, loader):
#   criterion = torch.nn.BCELoss()
#   #model.eval()
#   loss = 0
#   acc = 0

#   for data in loader:
#     data.x = data.x.to(device)
#     data.edge_index = data.edge_index.to(device)
#     data.y = data.y.to(device).float()
#     data.batch = data.batch.to(device)

#     out = model(data.x, data.edge_index, data.batch)
#     # print("Output shape:", out.shape)
#     # print("Target shape:", data.y.shape)

#     out = out.view(-1)

#     loss += criterion(out, data.y.float())/len(loader)
#     acc  += accuracy(out, data.y)/len(loader)

#   return loss, acc


# def accuracy(pred_y, y):
#   #print("Before Squeeze", pred_y)
#   pred = pred_y.squeeze()
#   #print("After Squeeze", pred)
#   pred = (pred >= 0.5).float()
#   #print("After thresholding", pred)
#   return (pred == y).sum().item()/len(y)





## Training

In [ ]:
def train(model, loader, lr, epochs,i):
  criterion = torch.nn.BCELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = lr)
  epochs = epochs
  model.train()

  train_accuracies = []
  val_accuracies = []
  best_val_acc = 0
  best_model_state = None
  for epoch in range(epochs+1):
    total_loss = 0
    acc = 0
    val_loss = 0
    val_acc = 0

    for data in loader:
      data.x = data.x.to(device)
      data.edge_index = data.edge_index.to(device)
      data.y = data.y.to(device).float()
      data.batch = data.batch.to(device)
      #data.y = data.y.view(-1,1)

      optimizer.zero_grad()
      out = model(data.x, data.edge_index, data.batch)
      out = out.view(-1)
      #print("Before view", out)
      loss = criterion(out, data.y)
      total_loss += loss/ len(loader)
      acc += accuracy(out, data.y)/len(loader)
      loss.backward()
      optimizer.step()

    val_loss, val_acc = test(model, val_loader)
    val_accuracies.append(val_acc )

    #print(len(train_accuracies),len(val_accuracies))
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      best_train_acc = acc
      best_epoch = epoch
      best_model_state = model.state_dict()


    train_accuracies.append(round(acc*100,2))
   # total_loss /= len(loader)
   # acc /= len(loader)


    if(epoch % 5 == 0):
      print(f'Epoch {epoch:>3} | Train Loss: {total_loss:.2f} | Train Acc: {acc*100:>.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc:.2f}%')
    #print(len(val_accuracies), len(train_accuracies))


  model_name = type(model).__name__

  torch.save(best_model_state, f'/content/drive/MyDrive/graph_data/Best_models/exp{i}_{model_name}.pth')
  model.load_state_dict(best_model_state)

  plt.figure(figsize=(8, 5))
  plt.plot(range(epochs + 1), train_accuracies, label='Training Accuracy')
  plt.plot(range(epochs + 1), val_accuracies, label='Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.title(f'Training vs. Validation Accuracy of {model_name}')
  plt.show()


  #return model, best_val_acc, best_train_acc, best_epoch
  return model, round(best_train_acc * 100, 2), round(best_val_acc * 100, 2), best_epoch
@torch.no_grad()
def test(model, loader):
  criterion = torch.nn.BCELoss()
  #model.eval()
  loss = 0
  acc = 0

  for data in loader:
    data.x = data.x.to(device)
    data.edge_index = data.edge_index.to(device)
    data.y = data.y.to(device).float()
    data.batch = data.batch.to(device)

    out = model(data.x, data.edge_index, data.batch)
    # print("Output shape:", out.shape)
    # print("Target shape:", data.y.shape)

    out = out.view(-1)

    loss += criterion(out, data.y.float())/len(loader)
    acc  += accuracy(out, data.y)/len(loader)

  return loss, round(acc *100, 2)


def accuracy(pred_y, y):
  #print("Before Squeeze", pred_y)
  pred = pred_y.squeeze()
  #print("After Squeeze", pred)
  pred = (pred >= 0.5).float()
  #print("After thresholding", pred)
  return (pred == y).sum().item()/len(y)

# Logs:
# the validation accuracy has been pushed to the epoch for loop, have to run the models again and check how the metrics are changing.
# Check if the best model is the one that is being returned.
# Perform all three experiments with the learning rate set to 0.01




In [ ]:
num_node_features = 9
batch_size = 8
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
gs = GraphSAGE(dim_h = 32).to(device)
model, train_acc, val_acc, epochs= train(gs, train_loader, 0.001, 100,1)
test_loss, test_acc = test(model, test_loader)

print(f'Test Loss: {test_loss} | Test Acc: {test_acc}%')
print(f'Training Accuracy: {train_acc}| Validation Accuracy: {val_acc}| Epoch: {epochs}')

In [ ]:
#torch.save(model, f"/content/drive/MyDrive/graph_data/best_graphSAGE.pth")

In [ ]:
c_matplot(model, test_loader, 2)

# Experiments

## Experiment - 01

*   Batch_size = 8
*   Learning Rate = 0.001

*   Epochs = 100

*   node_features = 9




In [ ]:
batch_size = 8
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
trained_models = []
testing_acc_1 = []
training_acc_1 = []
validation_acc_1 = []
epoch_1 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32*8).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.001, 100,1)
  trained_models.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_1.append(test_acc)
  training_acc_1.append(train_acc)
  validation_acc_1.append(val_acc)
  epoch_1.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_1, 'Test_acc': testing_acc_1, 'Val_acc':validation_acc_1, 'Epoch':epoch_1}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp1.csv')

In [ ]:
for i in trained_models:
  c_matplot(i, test_loader, 1)


## Experiment - 02
*   Batch_size = 16
*   Learning Rate = 0.001

*   Epochs = 100

*   node_features = 9


In [ ]:
batch_size = 16
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
trained_models_2 = []
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
testing_acc_2 = []
training_acc_2 = []
validation_acc_2 = []
epoch_2 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.001, 100,2)
  trained_models_2.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_2.append(test_acc)
  training_acc_2.append(train_acc)
  validation_acc_2.append(val_acc)
  epoch_2.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_2, 'Test_acc': testing_acc_2, 'Val_acc':validation_acc_2, 'Epoch': epoch_2}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp2.csv')

In [ ]:
for i in trained_models_2:
  c_matplot(i, test_loader,2)

## Experiment- 03

In [ ]:
batch_size = 32
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
trained_models_3 = []
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
testing_acc_3 = []
training_acc_3 = []
validation_acc_3 = []
epoch_3 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.001, 120,3)
  trained_models_3.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_3.append(test_acc)
  training_acc_3.append(train_acc)
  validation_acc_3.append(val_acc)
  epoch_3.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_3, 'Test_acc': testing_acc_3, 'Val_acc':validation_acc_3, 'Epoch':epoch_3}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp3.csv')

In [ ]:
for i in trained_models_3:
  c_matplot(i, test_loader,3)

## Experiment: 4

Learning rate is increased one order

In [ ]:
batch_size = 8
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
trained_models = []
testing_acc_1 = []
training_acc_1 = []
validation_acc_1 = []
epoch_1 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.01, 120,4)
  trained_models.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_1.append(test_acc)
  training_acc_1.append(train_acc)
  validation_acc_1.append(val_acc)
  epoch_1.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_1, 'Test_acc': testing_acc_1, 'Val_acc':validation_acc_1, 'Epoch':epoch_1}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp4.csv')

In [ ]:
for i in trained_models_:
  c_matplot(i, test_loader,4)

## Experiment: 5


In [ ]:
batch_size = 16
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
trained_models_2 = []
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
testing_acc_2 = []
training_acc_2 = []
validation_acc_2 = []
epoch_2 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.01, 120,5)
  trained_models_2.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_2.append(test_acc)
  training_acc_2.append(train_acc)
  validation_acc_2.append(val_acc)
  epoch_2.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_2, 'Test_acc': testing_acc_2, 'Val_acc':validation_acc_2, 'Epoch': epoch_2}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp5.csv')

In [ ]:
for i in trained_models_2:
  c_matplot(i, test_loader,5)

## Experiment:6

In [ ]:
batch_size = 32
train_loader, val_loader, test_loader = loader(batch_size = batch_size)
trained_models_3 = []
models = [GCN, GCN_m, GIN, GAT, GraphSAGE]
testing_acc_3 = []
training_acc_3 = []
validation_acc_3 = []
epoch_3 = []
for model in models:
  num_node_features = 9
  model = model(dim_h = 32).to(device)
  model, train_acc, val_acc, epochs = train(model, train_loader, 0.01, 200,6)
  trained_models_3.append(model)
  test_loss, test_acc = test(model, test_loader)
  testing_acc_3.append(test_acc)
  training_acc_3.append(train_acc)
  validation_acc_3.append(val_acc)
  epoch_3.append(epochs)
  print(f'Test Loss: {test_loss : 2f} | Test Acc: {test_acc: 2f}%')

dict = {'Train_acc': training_acc_3, 'Test_acc': testing_acc_3, 'Val_acc':validation_acc_3, 'Epoch':epoch_3}
df = pd.DataFrame(dict)
df.to_csv('/content/drive/MyDrive/graph_data/Metrics/exp6.csv')

In [ ]:
for i in trained_models_3:
  c_matplot(i, test_loader,6)